In [1]:
import cv2

In [ ]:
import cv2
import numpy as np

logo = cv2.imread("./image.png", cv2.IMREAD_GRAYSCALE)
shirt = cv2.imread("./shirts/image.png", cv2.IMREAD_GRAYSCALE)

if logo is None or shirt is None:
    print("Error loading images")
    exit()

# SIFT with your custom params
sift = cv2.SIFT_create(
    nOctaveLayers=5,
    sigma=1.414
)

kp_logo, des_logo = sift.detectAndCompute(logo, None)
kp_shirt, des_shirt = sift.detectAndCompute(shirt, None)

# --- 1) Match with KNN (no crossCheck here) ---
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
knn_matches = bf.knnMatch(des_logo, des_shirt, k=2)

# --- 2) Lowe's ratio test ---
good = []
ratio_thresh = 0.75
for m, n in knn_matches:
    if m.distance < ratio_thresh * n.distance:
        good.append(m)

print("Good matches:", len(good))

MIN_MATCH_COUNT = 0  # you can tweak this

if len(good) >= MIN_MATCH_COUNT:
    # --- 3) Prepare points for findHomography ---
    src_pts = np.float32([kp_logo[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp_shirt[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

    # --- 4) Compute Homography using RANSAC ---
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    matchesMask = mask.ravel().tolist() if mask is not None else None

    print("Homography:\n", H)

    # --- 5) Draw inlier matches only ---
    result = cv2.drawMatches(
        logo, kp_logo,
        shirt, kp_shirt,
        good, None,
        matchesMask=matchesMask,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )

    cv2.namedWindow("RANSAC Matches", cv2.WINDOW_NORMAL)
    cv2.imshow("RANSAC Matches", result)
    cv2.resizeWindow("RANSAC Matches", 1200, 800)

    # --- 6) Optional: draw logo projection (bounding box) on shirt ---
    if H is not None:
        h, w = logo.shape
        # Logo corners
        logo_corners = np.float32([
            [0, 0],
            [0, h - 1],
            [w - 1, h - 1],
            [w - 1, 0]
        ]).reshape(-1, 1, 2)

        # Project corners into shirt image
        projected_corners = cv2.perspectiveTransform(logo_corners, H)

        # Convert shirt to color to draw colored polygon
        shirt_color = cv2.cvtColor(shirt, cv2.COLOR_GRAY2BGR)
        shirt_with_box = cv2.polylines(
            shirt_color,
            [np.int32(projected_corners)],
            isClosed=True,
            color=(0, 255, 0),
            thickness=3,
            lineType=cv2.LINE_AA
        )

        cv2.namedWindow("Logo on Shirt", cv2.WINDOW_NORMAL)
        cv2.imshow("Logo on Shirt", shirt_with_box)
        cv2.resizeWindow("Logo on Shirt", 800, 800)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

else:
    print(f"Not enough matches for RANSAC: {len(good)} / {MIN_MATCH_COUNT}")


Good matches: 16
Homography:
 [[ 8.28444829e-01 -1.49648201e+00  2.01712238e+02]
 [ 3.15802380e-01 -8.96780059e-01  2.30443086e+02]
 [ 1.59485923e-03 -5.06620229e-03  1.00000000e+00]]


In [16]:
import cv2
import numpy as np

def preprocess_image(img_path, use_segmentation=True):
    """
    Preprocess image with optional mean shift filtering and segmentation.
    Returns both grayscale and preprocessed version.
    """
    img = cv2.imread(img_path)
    if img is None:
        return None, None
    
    if use_segmentation:
        # Apply mean shift filtering for segmentation
        filtered = cv2.pyrMeanShiftFiltering(img, sp=20, sr=40)
        
        # Convert to grayscale and apply Otsu's thresholding
        gray = cv2.cvtColor(filtered, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)
        
        return gray, thresh
    else:
        # Simple grayscale conversion
        gray = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) if len(img.shape) == 2 else cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return gray, gray

def detect_logo_on_shirt(logo_path, shirt_path, show_visualization=True):
    """
    Detect and match a logo on a shirt using SIFT and RANSAC.
    
    Args:
        logo_path: Path to the logo image
        shirt_path: Path to the shirt/product image
        show_visualization: Whether to display results
    
    Returns:
        dict with match results including number of matches and homography matrix
    """
    # Load images
    logo = cv2.imread(logo_path, cv2.IMREAD_GRAYSCALE)
    shirt_original = cv2.imread(shirt_path)
    
    if logo is None or shirt_original is None:
        print("Error loading images")
        return None
    
    # Preprocess shirt image with segmentation
    shirt_gray, shirt_segmented = preprocess_image(shirt_path, use_segmentation=True)
    
    if show_visualization:
        cv2.imshow("1. Original Shirt", shirt_original)
        cv2.imshow("2. Segmented Shirt", shirt_segmented)
        cv2.imshow("3. Logo to Match", logo)
        cv2.waitKey(1000)
    
    # Use segmented image for better feature detection
    shirt = shirt_segmented
    
    # SIFT feature detection
    sift = cv2.SIFT_create(
        nOctaveLayers=5,
        sigma=1.414
    )
    
    kp_logo, des_logo = sift.detectAndCompute(logo, None)
    kp_shirt, des_shirt = sift.detectAndCompute(shirt, None)
    
    print(f"Logo keypoints: {len(kp_logo)}")
    print(f"Shirt keypoints: {len(kp_shirt)}")
    
    # Match with KNN
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    knn_matches = bf.knnMatch(des_logo, des_shirt, k=2)
    
    # Lowe's ratio test
    good = []
    ratio_thresh = 0.75
    for pair in knn_matches:
        if len(pair) == 2:
            m, n = pair
            if m.distance < ratio_thresh * n.distance:
                good.append(m)
    
    print(f"Good matches after ratio test: {len(good)}")
    
    MIN_MATCH_COUNT = 10
    result_dict = {
        'matches': len(good),
        'success': False,
        'homography': None,
        'inliers': 0
    }
    
    if len(good) >= MIN_MATCH_COUNT:
        # Prepare points for homography
        src_pts = np.float32([kp_logo[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_shirt[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
        
        # Compute Homography using RANSAC
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        
        if H is not None and mask is not None:
            matchesMask = mask.ravel().tolist()
            inliers = sum(matchesMask)
            
            result_dict['success'] = True
            result_dict['homography'] = H
            result_dict['inliers'] = inliers
            
            print(f"RANSAC inliers: {inliers}/{len(good)}")
            print("Homography matrix:\n", H)
            
            if show_visualization:
                # Draw matches
                result = cv2.drawMatches(
                    logo, kp_logo,
                    shirt, kp_shirt,
                    good, None,
                    matchesMask=matchesMask,
                    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
                )
                
                cv2.namedWindow("4. RANSAC Matches", cv2.WINDOW_NORMAL)
                cv2.imshow("4. RANSAC Matches", result)
                cv2.resizeWindow("4. RANSAC Matches", 1200, 800)
                
                # Draw logo bounding box on original shirt
                h, w = logo.shape
                logo_corners = np.float32([
                    [0, 0],
                    [0, h - 1],
                    [w - 1, h - 1],
                    [w - 1, 0]
                ]).reshape(-1, 1, 2)
                
                projected_corners = cv2.perspectiveTransform(logo_corners, H)
                
                shirt_with_box = shirt_original.copy()
                shirt_with_box = cv2.polylines(
                    shirt_with_box,
                    [np.int32(projected_corners)],
                    isClosed=True,
                    color=(0, 255, 0),
                    thickness=3,
                    lineType=cv2.LINE_AA
                )
                
                cv2.namedWindow("5. Logo Detection Result", cv2.WINDOW_NORMAL)
                cv2.imshow("5. Logo Detection Result", shirt_with_box)
                cv2.resizeWindow("5. Logo Detection Result", 800, 800)
                
                cv2.waitKey(0)
                cv2.destroyAllWindows()
        else:
            print("Homography computation failed")
    else:
        print(f"Not enough matches for RANSAC: {len(good)} / {MIN_MATCH_COUNT}")
        if show_visualization:
            cv2.destroyAllWindows()
    
    return result_dict

def batch_logo_detection(logo_path, shirt_paths):
    """
    Test a logo against multiple shirt images.
    
    Args:
        logo_path: Path to the logo image
        shirt_paths: List of paths to shirt images
    
    Returns:
        List of results for each shirt
    """
    results = []
    for i, shirt_path in enumerate(shirt_paths):
        print(f"\n{'='*60}")
        print(f"Testing shirt {i+1}/{len(shirt_paths)}: {shirt_path}")
        print(f"{'='*60}")
        
        result = detect_logo_on_shirt(logo_path, shirt_path, show_visualization=False)
        results.append({
            'shirt_path': shirt_path,
            'result': result
        })
        
        if result and result['success']:
            print(f"✓ LOGO DETECTED - {result['inliers']} inliers")
        else:
            print(f"✗ NO LOGO DETECTED")
    
    return results

# Main execution
if __name__ == "__main__":
    # Single image detection
    logo_path = "./image.png"
    shirt_path = "./shirts/image.png"
    
    print("Starting brand logo detection...")
    result = detect_logo_on_shirt(logo_path, shirt_path, show_visualization=True)
    
    if result:
        if result['success']:
            print(f"\n✓ SUCCESS: Logo detected with {result['inliers']} RANSAC inliers")
        else:
            print(f"\n✗ FAILED: Logo not detected (only {result['matches']} matches)")
    
    # Uncomment for batch processing:
    # shirt_paths = [
    #     "./shirts/image1.png",
    #     "./shirts/image2.png",
    #     "./shirts/image3.png"
    # ]
    # batch_results = batch_logo_detection(logo_path, shirt_paths)

Starting brand logo detection...
Logo keypoints: 589
Shirt keypoints: 617
Good matches after ratio test: 63
RANSAC inliers: 33/63
Homography matrix:
 [[ 2.36965735e+00 -1.07740640e+01  2.56683692e+02]
 [ 2.75837980e+00 -1.25238668e+01  2.97046512e+02]
 [ 9.14455047e-03 -4.16980519e-02  1.00000000e+00]]

✓ SUCCESS: Logo detected with 33 RANSAC inliers


Status: UNKNOWN
Brand: Unknown Brand
Confidence: 0.00
Results saved to 'detection_result.jpg'
